In [1]:
import pandas as pd
import json
import os
import requests
from datetime import datetime, time

In [2]:

file1 = 'rawdata/불법주정차 신고현황(21년9월29일_22년10월31일).csv'
file2 = 'rawdata/불법주정차 신고현황(22.11월~23.10월).csv'
file3 = 'rawdata/불법주정차 신고현황(23년11월1일_24년3월13일).csv'

df_reports_1 = pd.read_csv(file1, encoding='utf-8')
df_reports_2 = pd.read_csv(file2, encoding='utf-8')
df_reports_3 = pd.read_csv(file3, encoding='utf-8')


In [3]:
display(df_reports_1.head())

,민원접수일,민원접수시간,주소,경도,위도
0,20210929,00:05:00,서울 강서구 화곡로66길 130,126.859911,37.555537
1,20210929,00:38:00,중구 다산로48길 33 단우물어린이집앞,127.018013,37.567483
2,20210929,00:38:00,서울특별시 중구 마장로 19,127.012793,37.569183
3,20210929,00:50:00,서울특별시 강서구 까치산로15길 21,126.847125,37.547040
4,20210929,01:15:00,서울특별시 중구 다산로 293,127.015703,37.569085


In [4]:
print(df_reports_1.shape)
print(df_reports_2.shape)
print(df_reports_3.shape)

(1110253, 5)
(1466617, 5)
(493658, 5)


In [5]:
#Add reset_index function
df_reports = pd.concat([df_reports_3, df_reports_1, df_reports_2], axis=0).reset_index(drop=True)
df_reports.shape

(3070528, 5)

In [6]:
file4 = 'rawdata/서울시 공영주차장 안내 정보.csv'
df_park_info = pd.read_csv(file4, encoding='cp949')

In [7]:
display(df_park_info.columns)

Index(['주차장코드', '주차장명', '주소', '주차장 종류', '주차장 종류명', '운영구분', '운영구분명', '전화번호',
       '주차현황 정보 제공여부', '주차현황 정보 제공여부명', '총 주차면', '유무료구분', '유무료구분명', '야간무료개방여부',
       '야간무료개방여부명', '평일 운영 시작시각(HHMM)', '평일 운영 종료시각(HHMM)', '주말 운영 시작시각(HHMM)',
       '주말 운영 종료시각(HHMM)', '공휴일 운영 시작시각(HHMM)', '공휴일 운영 종료시각(HHMM)',
       '최종데이터 동기화 시간', '토요일 유,무료 구분', '토요일 유,무료 구분명', '공휴일 유,무료 구분',
       '공휴일 유,무료 구분명', '월 정기권 금액', '노상 주차장 관리그룹번호', '기본 주차 요금',
       '기본 주차 시간(분 단위)', '추가 단위 요금', '추가 단위 시간(분 단위)', '버스 기본 주차 요금',
       '버스 기본 주차 시간(분 단위)', '버스 추가 단위 시간(분 단위)', '버스 추가 단위 요금', '일 최대 요금',
       '위도', '경도'],
      dtype='object')

In [8]:
df_park_info.head()

,주차장코드,주차장명,주소,주차장 종류,주차장 종류명,운영구분,운영구분명,전화번호,주차현황 정보 제공여부,주차현황 정보 제공여부명,...,기본 주차 시간(분 단위),추가 단위 요금,추가 단위 시간(분 단위),버스 기본 주차 요금,버스 기본 주차 시간(분 단위),버스 추가 단위 시간(분 단위),버스 추가 단위 요금,일 최대 요금,위도,경도
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,NW,노외 주차장,1,시간제 주차장,NaN,0,미연계중,...,NaN,300.0,10.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,NW,노외 주차장,1,시간제 주차장,02-2289-6735,0,미연계중,...,5.0,150.0,5.0,0.0,0.0,0.0,0.0,11000.0,NaN,NaN
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2,NW,노외 주차장,1,시간제 주차장,02-2204-7970,2,20~120분이내 연계데이터 존재(정보수집중),...,5.0,50.0,5.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,NW,노외 주차장,1,시간제 주차장,02-2677-1401,0,미연계중,...,5.0,50.0,5.0,0.0,0.0,0.0,0.0,0.0,37.515205,126.915116
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,NW,노외 주차장,1,시간제 주차장,02-2677-1401,1,현재~20분이내 연계데이터 존재(현재 주차대수 표현),...,5.0,150.0,5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [18]:

# Rename columns
df_park_info = df_park_info.rename(columns={
    '주차장 종류': '주차장종류',
    '총 주차면': '총주차면',
    '유무료구분': '평일유료',
    '평일 운영 시작시각(HHMM)': '평일시작',
    '평일 운영 종료시각(HHMM)': '평일종료',
    '주말 운영 시작시각(HHMM)': '토요일시작',
    '주말 운영 종료시각(HHMM)': '토요일종료',
    '공휴일 운영 시작시각(HHMM)': '공휴일시작',
    '공휴일 운영 종료시각(HHMM)': '공휴일종료',
    '토요일 유,무료 구분': '토요일유료',
    '공휴일 유,무료 구분': '공휴일유료',
    '기본 주차 요금': '기본주차요금',
    '기본 주차 시간(분 단위)': '기본주차시간',
    '추가 단위 요금': '추가단위요금',
    '추가 단위 시간(분 단위)': '추가단위시간'
})

# Define the desired column order
new_columns = ['주소', '주차장종류', '운영구분', '총주차면', '평일유료', '토요일유료', '공휴일유료', '평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료', '기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간', '경도', '위도']

# Reorder the columns in the DataFrame
df_parks = df_park_info[new_columns]

display(df_parks.head())
# display(df_parks.tail())


,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도
0,도봉구 창동 24-0,NW,1,71.0,Y,N,N,900,1900,900,1900,900,1900,0.0,NaN,300.0,10.0,NaN,NaN
1,노원구 상계동 770-2,NW,1,177.0,Y,N,N,0,2400,0,2400,0,2400,150.0,5.0,150.0,5.0,NaN,NaN
2,성동구 마장동 463-2,NW,1,54.0,Y,N,N,0,2400,0,2400,0,2400,50.0,5.0,50.0,5.0,NaN,NaN
3,영등포구 신길동 184-3,NW,1,98.0,Y,N,N,0,2400,0,2400,0,2400,50.0,5.0,50.0,5.0,126.915116,37.515205
4,영등포구 당산동3가 385-0,NW,1,190.0,Y,N,N,0,2400,0,2400,0,2400,150.0,5.0,150.0,5.0,NaN,NaN


In [19]:
# Extract rows where '운영구분' is either '1' or '3'
df_park_type1 = df_parks[df_parks['운영구분'] ==  1]
df_park_type3 = df_parks[df_parks['운영구분'] == 3]

#df_park_type1.shape >> (6210,19)
#'시간제 주차장' : 6210개
#df_park_type3.shape >> (1708,19)
#'시간제 + 거주자 주차장' : 1708개

df_parks = pd.concat([df_park_type1, df_park_type3], axis=0).reset_index(drop=True)

# Verify that all '운영구분' values are valid
normaltype = [1, 3]
for i in range(df_parks.shape[0]):
    if df_parks.loc[i,]['운영구분'] not in normaltype: print('error')

In [20]:
df_parks

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도
0,도봉구 창동 24-0,NW,1,71.0,Y,N,N,900,1900,900,1900,900,1900,0.0,NaN,300.0,10.0,NaN,NaN
1,노원구 상계동 770-2,NW,1,177.0,Y,N,N,0,2400,0,2400,0,2400,150.0,5.0,150.0,5.0,NaN,NaN
2,성동구 마장동 463-2,NW,1,54.0,Y,N,N,0,2400,0,2400,0,2400,50.0,5.0,50.0,5.0,NaN,NaN
3,영등포구 신길동 184-3,NW,1,98.0,Y,N,N,0,2400,0,2400,0,2400,50.0,5.0,50.0,5.0,126.915116,37.515205
4,영등포구 당산동3가 385-0,NW,1,190.0,Y,N,N,0,2400,0,2400,0,2400,150.0,5.0,150.0,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7913,강북구 수유동 535-76,NS,3,1.0,Y,N,N,900,1900,900,1500,900,1800,200.0,5.0,200.0,5.0,NaN,NaN
7914,강북구 수유동 535-76,NS,3,1.0,Y,N,N,900,1900,900,1500,900,1800,200.0,5.0,200.0,5.0,NaN,NaN
7915,강북구 수유동 49-27,NS,3,1.0,Y,N,N,900,1900,900,1500,0,0,200.0,5.0,200.0,5.0,NaN,NaN
7916,중랑구 면목동 146-1,NW,3,199.0,Y,N,N,0,2400,0,2400,0,2400,100.0,5.0,100.0,5.0,NaN,NaN


In [21]:
# Replace NaN values in '기본주차요금', '기본주차시간', '추가단위요금', and '추가단위시간' columns with 0.0
df_parks['기본주차요금'] = df_parks['기본주차요금'].fillna(value = 0.0)
df_parks['기본주차시간'] = df_parks['기본주차시간'].fillna(value = 0.0)
df_parks['추가단위요금'] = df_parks['추가단위요금'].fillna(value = 0.0)
df_parks['추가단위시간'] = df_parks['추가단위시간'].fillna(value = 0.0)


# Verify that all values in '기본주차요금', '기본주차시간', '추가단위요금', and '추가단위시간' columns are valid
print(set(list(df_parks['기본주차요금'])))
print(set(list(df_parks['기본주차시간'])))
print(set(list(df_parks['추가단위요금'])))
print(set(list(df_parks['추가단위시간'])))

print(df_parks['기본주차요금'].isna().sum())
print(df_parks['기본주차시간'].isna().sum())
print(df_parks['추가단위요금'].isna().sum())
print(df_parks['추가단위시간'].isna().sum())

{0.0, 130.0, 260.0, 900.0, 400.0, 150.0, 2200.0, 160.0, 800.0, 4000.0, 550.0, 680.0, 170.0, 300.0, 430.0, 1200.0, 50.0, 440.0, 3000.0, 700.0, 190.0, 320.0, 65.0, 450.0, 2500.0, 200.0, 330.0, 75.0, 80.0, 2000.0, 600.0, 1500.0, 220.0, 350.0, 100.0, 1000.0, 360.0, 620.0, 750.0, 500.0, 120.0, 250.0}
{0.0, 420.0, 5.0, 10.0, 240.0, 20.0, 60.0, 540.0, 30.0}
{0.0, 130.0, 260.0, 900.0, 650.0, 400.0, 150.0, 160.0, 800.0, 4000.0, 550.0, 680.0, 170.0, 300.0, 430.0, 1200.0, 50.0, 440.0, 3000.0, 700.0, 190.0, 320.0, 65.0, 450.0, 4800.0, 200.0, 330.0, 1100.0, 80.0, 600.0, 220.0, 1500.0, 350.0, 100.0, 1000.0, 360.0, 620.0, 750.0, 500.0, 120.0, 250.0}
{0.0, 5.0, 10.0, 15.0, 60.0, 540.0, 30.0}
0
0
0
0


In [22]:
df_parks

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도
0,도봉구 창동 24-0,NW,1,71.0,Y,N,N,900,1900,900,1900,900,1900,0.0,0.0,300.0,10.0,NaN,NaN
1,노원구 상계동 770-2,NW,1,177.0,Y,N,N,0,2400,0,2400,0,2400,150.0,5.0,150.0,5.0,NaN,NaN
2,성동구 마장동 463-2,NW,1,54.0,Y,N,N,0,2400,0,2400,0,2400,50.0,5.0,50.0,5.0,NaN,NaN
3,영등포구 신길동 184-3,NW,1,98.0,Y,N,N,0,2400,0,2400,0,2400,50.0,5.0,50.0,5.0,126.915116,37.515205
4,영등포구 당산동3가 385-0,NW,1,190.0,Y,N,N,0,2400,0,2400,0,2400,150.0,5.0,150.0,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7913,강북구 수유동 535-76,NS,3,1.0,Y,N,N,900,1900,900,1500,900,1800,200.0,5.0,200.0,5.0,NaN,NaN
7914,강북구 수유동 535-76,NS,3,1.0,Y,N,N,900,1900,900,1500,900,1800,200.0,5.0,200.0,5.0,NaN,NaN
7915,강북구 수유동 49-27,NS,3,1.0,Y,N,N,900,1900,900,1500,0,0,200.0,5.0,200.0,5.0,NaN,NaN
7916,중랑구 면목동 146-1,NW,3,199.0,Y,N,N,0,2400,0,2400,0,2400,100.0,5.0,100.0,5.0,NaN,NaN


In [23]:
# Duplicate Parking Lot Merging Process

# Sort by '주소'
df_parks = df_parks.sort_values(by="주소").reset_index(drop=True)


# Merge identical parking lots and update total parking spaces
count = 1                                                                   #중복 주차장의 총주차면(initial value : 1)
checklist = ['주소', '기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간']       # 동일 주차장(일 경우 총주차면 결합)을 판단하는 columns list
to_drop = []

df_parks[checklist] = df_parks[checklist].astype(str).apply(lambda col: col.str.strip())

for i in range(df_parks.shape[0]):
    if i == 0: continue
    #df.reset_index(drop=True)
    # Parking lots with type 'NW'
    # 지금까지 주차장의 중복이 지속되어왔을 경우 중복주차장 결합 처리
    if (df_parks.iloc[i]['주차장종류'] == 'NW') and (count>1):
        idx = df_parks.index[i - count]
        df_parks.at[idx, '총주차면'] = count
        to_drop = to_drop + list(range(idx+1,idx+count))
        count = 1
    elif (df_parks.iloc[i]['주차장종류']== 'NW') and (count ==1): 
        continue
    
    # Parking lots with type 'NS'
    # 현재 반복에서까지 중복주차장이 발견되지 않았을 경우 -> continue
    # 중복주차장이 발견되고 있으면서 이번 주차장도 중복 주차장을 경우  -> count = count+총주차면
    # 중복주차장이 발견되고 있었지만 이번 주차장은 중복이 아닌 경우 -> 이전주차장까지 중복주차장 결합 처리
    else : 
        isIdentical = True
        for field in checklist:
            isIdentical = True
            for field in checklist:
                if df_parks.iloc[i][field] != df_parks.iloc[i-1][field]:
                    isIdentical = False
                    break
        if isIdentical : 
            count = count + int(df_parks.iloc[i]['총주차면'])
        elif not(isIdentical) and (count>1) :
            idx = df_parks.index[i - count]
            df_parks.at[idx, '총주차면'] = count
            to_drop = to_drop + list(range(idx+1, idx+count))
            count = 1
        else :
            continue
            
if count > 1:
    last_idx = df_parks.index[-count]
    df_parks.at[last_idx, '총주차면'] = count
    to_drop += list(range(df_parks.index[-count + 1], df_parks.index[-1] + 1))
            
df_parks = df_parks.drop(to_drop).reset_index(drop=True)

In [24]:
# Calculate 1-hour parking fee based on base and extra fee rules, and add it as a new column named '1시간 요금'

length = df_parks.shape[0]
feelist = [0 for i in range(length)]

for i in range(length):
    basefee = float(df_parks['기본주차요금'][i])
    basetime = float(df_parks['기본주차시간'][i])
    extrafee = float(df_parks['추가단위요금'][i])
    extratime = float(df_parks['추가단위시간'][i])
    
    if basetime == 0 :
        if extratime == 0 : feelist[i] = 0
        elif 60%extratime == 0 : feelist[i] = (60/extratime)*extrafee
        else : feelist[i] = ((60/extratime)+1)*extrafee
    elif ((basetime > 0) & (basetime < 60)) :
        if extratime == 0 : feelist[i] = basefee
        elif 60%extratime == 0 : feelist[i] = ((60-basetime)/extratime)*extrafee + basefee
        else : feelist[i] = (((60-basetime)/extratime)+1)*extrafee + basefee
    else :
        feelist[i] = basefee
    
df_parks['1시간 요금'] = feelist
df_parks

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,900,1900,0,0,0,0,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,0,2400,0,2400,0,2400,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
2,강남구 개포동 1273-0,NW,1,194.0,Y,N,N,0,2400,0,2400,0,2400,200.0,5.0,200.0,5.0,127.052365,37.475646,2400.0
3,강남구 개포동 13-2,NW,1,168.0,Y,N,N,0,2400,0,2400,0,2400,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
4,강남구 개포동 183-2,NW,1,149.0,Y,N,N,0,2400,0,2400,0,2400,200.0,5.0,200.0,5.0,127.066238,37.489490,2400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,중랑구 중화동 29-2,NW,1,130.0,Y,N,N,0,2400,0,2400,0,2400,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
1517,중랑구 중화동 304-36,NS,3,4.0,Y,N,N,900,1900,900,1500,0,0,250.0,5.0,250.0,5.0,NaN,NaN,3000.0
1518,중랑구 중화동 61-2,NS,1,1.0,Y,N,Y,900,1900,900,1500,0,0,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
1519,중랑구 중화동 61-4,NS,3,1.0,Y,N,N,900,1800,0,0,0,0,600.0,60.0,300.0,30.0,NaN,NaN,600.0


In [25]:
def parse_time(value):

    if pd.isna(value):
        return None

    time_str = str(int(value)).zfill(4) if isinstance(value, (int, float)) else str(value)

    # Special-case “2400” or “24:00[:00]” as midnight
    if time_str in ("2400", "24:00", "24:00:00"):
        return time(0, 0)

    try:
        # Try parsing as HHMM
        return pd.to_datetime(time_str, format="%H%M").time()

    except:
        return None

In [26]:
time_columns = ['평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료']
for col in time_columns:
    df_parks[col] = df_parks[col].apply(parse_time).copy()

display(df_parks.head())

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
2,강남구 개포동 1273-0,NW,1,194.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.052365,37.475646,2400.0
3,강남구 개포동 13-2,NW,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
4,강남구 개포동 183-2,NW,1,149.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.066238,37.489490,2400.0


In [27]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_parks[df_parks.drop(columns=['경도', '위도']).isnull().any(axis=1)]

# Display rows with missing values
print("Rows with missing values")
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_parks_dropped = df_parks.drop(missing_rows.index)


Rows with missing values


,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
54,강남구 역삼동 1-1,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1016,송파구 가락동 156-11,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1027,송파구 마천동 27-3,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1028,송파구 마천동 30-0,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1031,송파구 문정동 114-4,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1042,송파구 문정동 640-9,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1056,송파구 삼전동 20-0,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1093,송파구 장지동 770-0,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1095,송파구 풍납동 159-9,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1097,송파구 풍납동 201-7,NW,1,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,0.0


In [28]:
df_parks_dropped = df_parks_dropped.reset_index(drop=True)
# Display the updated DataFrame
display(df_parks_dropped)


,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
2,강남구 개포동 1273-0,NW,1,194.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.052365,37.475646,2400.0
3,강남구 개포동 13-2,NW,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
4,강남구 개포동 183-2,NW,1,149.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.066238,37.489490,2400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,중랑구 중화동 29-2,NW,1,130.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
1490,중랑구 중화동 304-36,NS,3,4.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,NaN,NaN,3000.0
1491,중랑구 중화동 61-2,NS,1,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
1492,중랑구 중화동 61-4,NS,3,1.0,Y,N,N,09:00:00,18:00:00,00:00:00,00:00:00,00:00:00,00:00:00,600.0,60.0,300.0,30.0,NaN,NaN,600.0


In [29]:
# Find rows in df_parks_dropped with missing coordinates
df_parks_missing = df_parks_dropped[df_parks_dropped.isnull().any(axis=1)]

# Drop these rows from df_parks_dropped
df_parks_dropped = df_parks_dropped.drop(df_parks_missing.index)

# Display the DataFrame with missing rows and the cleaned DataFrame
display(df_parks_missing)
display(df_parks_dropped)

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
3,강남구 개포동 13-2,NW,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
6,강남구 개포동 567-23,NW,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,NaN,NaN,2400.0
7,강남구 논현동 168-0,NW,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,300.0,5.0,300.0,5.0,NaN,NaN,3600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,중랑구 중화동 29-2,NW,1,130.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
1490,중랑구 중화동 304-36,NS,3,4.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,NaN,NaN,3000.0
1491,중랑구 중화동 61-2,NS,1,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,NaN,NaN,1200.0
1492,중랑구 중화동 61-4,NS,3,1.0,Y,N,N,09:00:00,18:00:00,00:00:00,00:00:00,00:00:00,00:00:00,600.0,60.0,300.0,30.0,NaN,NaN,600.0


,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
2,강남구 개포동 1273-0,NW,1,194.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.052365,37.475646,2400.0
4,강남구 개포동 183-2,NW,1,149.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.066238,37.489490,2400.0
5,강남구 개포동 186-7,NS,1,54.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,300.0,5.0,600.0,5.0,127.067813,37.488516,6900.0
8,강남구 논현동 279-22 언북중학교 뒷길,NS,1,14.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,400.0,5.0,800.0,5.0,127.033290,37.519800,9200.0
10,강남구 논현동 280-1,NS,1,7.0,Y,Y,Y,10:00:00,20:00:00,00:00:00,00:00:00,00:00:00,00:00:00,400.0,5.0,800.0,5.0,127.020223,37.515702,9200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,중구 흥인동 162-1 0,NS,1,20.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,500.0,5.0,500.0,5.0,127.017298,37.570006,6000.0
1460,중랑구 망우동 486-10,NS,1,56.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,127.099071,37.598946,3000.0
1463,중랑구 면목동 1-4,NW,1,119.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,0.0,0.0,127.089299,37.574183,100.0
1470,중랑구 면목동 168-2,NW,1,567.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.081451,37.579494,1200.0


In [30]:
# Function to get coordinates for a given address
def get_coordinates(address):
    apiurl = "https://api.vworld.kr/req/address?"
    apikey =  "D9C1CFC9-0FD3-3304-A3E7-131C33A91190"

    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",
        "key": apikey
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            point_info = data['response']['result']['point']
            return point_info['x'], point_info['y']
    return None, None


In [31]:
# Geocode missing coordinates
df_parks_missing[['경도', '위도']] = df_parks_missing['주소'].apply(lambda addr: pd.Series(get_coordinates(addr)))

# Display the updated dataframe
df_parks_missing

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.06647710446097,37.477263060380004,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.04821844306556,37.48149637668632,2400.0
3,강남구 개포동 13-2,NW,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.0793065206814,37.49493847412162,1200.0
6,강남구 개포동 567-23,NW,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.0658354146965,37.47788845798715,2400.0
7,강남구 논현동 168-0,NW,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,300.0,5.0,300.0,5.0,127.02629013795874,37.50817148532231,3600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,중랑구 중화동 29-2,NW,1,130.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.08164412157379,37.602722082374044,1200.0
1490,중랑구 중화동 304-36,NS,3,4.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,127.07849353699041,37.59955354159165,3000.0
1491,중랑구 중화동 61-2,NS,1,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.0721740315793,37.59513121414587,1200.0
1492,중랑구 중화동 61-4,NS,3,1.0,Y,N,N,09:00:00,18:00:00,00:00:00,00:00:00,00:00:00,00:00:00,600.0,60.0,300.0,30.0,127.07095279420147,37.5963884664176,600.0


In [32]:
df_parks_combined = pd.concat([df_parks_missing, df_parks_dropped], ignore_index=True)

# Display the combined DataFrame
display(df_parks_combined)

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.06647710446097,37.477263060380004,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.04821844306556,37.48149637668632,2400.0
2,강남구 개포동 13-2,NW,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.0793065206814,37.49493847412162,1200.0
3,강남구 개포동 567-23,NW,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.0658354146965,37.47788845798715,2400.0
4,강남구 논현동 168-0,NW,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,300.0,5.0,300.0,5.0,127.02629013795874,37.50817148532231,3600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,중구 흥인동 162-1 0,NS,1,20.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,500.0,5.0,500.0,5.0,127.017298,37.570006,6000.0
1490,중랑구 망우동 486-10,NS,1,56.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,127.099071,37.598946,3000.0
1491,중랑구 면목동 1-4,NW,1,119.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,0.0,0.0,127.089299,37.574183,100.0
1492,중랑구 면목동 168-2,NW,1,567.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.081451,37.579494,1200.0


In [33]:
# Display rows with missing values
missing_rows = df_parks_combined[df_parks_combined.isnull().any(axis=1)]
display(missing_rows.head())

# Drop rows with coordinate still missing after geocoding
df_parks_cleaned = df_parks_combined.dropna()

# Display the cleaned DataFrame
display(df_parks_cleaned)

,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
56,강북구 미아동 137-19,NS,3,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,None,None,2400.0
78,강북구 수유동 55-53,NW,1,116.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,None,None,1200.0
120,강서구 화곡동 980-19,NS,1,2.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,None,None,3000.0
121,강서구 화곡동 980-19,NS,1,3.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,None,None,3000.0
122,강서구 화곡동 980-19,NS,1,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,None,None,3000.0


,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,기본주차요금,기본주차시간,추가단위요금,추가단위시간,경도,위도,1시간 요금
0,강남구 개포동 126-2,NW,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.06647710446097,37.477263060380004,2400.0
1,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.04821844306556,37.48149637668632,2400.0
2,강남구 개포동 13-2,NW,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.0793065206814,37.49493847412162,1200.0
3,강남구 개포동 567-23,NW,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,200.0,5.0,200.0,5.0,127.0658354146965,37.47788845798715,2400.0
4,강남구 논현동 168-0,NW,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,300.0,5.0,300.0,5.0,127.02629013795874,37.50817148532231,3600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,중구 흥인동 162-1 0,NS,1,20.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,500.0,5.0,500.0,5.0,127.017298,37.570006,6000.0
1490,중랑구 망우동 486-10,NS,1,56.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,250.0,5.0,250.0,5.0,127.099071,37.598946,3000.0
1491,중랑구 면목동 1-4,NW,1,119.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,0.0,0.0,127.089299,37.574183,100.0
1492,중랑구 면목동 168-2,NW,1,567.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,100.0,5.0,100.0,5.0,127.081451,37.579494,1200.0


In [34]:
df_parks_cleaned.to_csv('parks.csv', index=False, encoding='utf-8-sig')

In [35]:
df_reports

,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
3070523,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
3070524,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
3070525,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
3070526,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [36]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_reports[df_reports.drop(columns=['경도', '위도']).isnull().any(axis=1)]

# Display rows with missing values
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_reports_dropped = df_reports.drop(missing_rows.index)

# Display the updated DataFrame
display(df_reports_dropped)

,민원접수일,민원접수시간,주소,경도,위도
2727,20231101,18:49:00,NaN,127.067222,37.664983
3092,20231101,19:51:00,NaN,127.067170,37.664344
3136,20231101,20:02:00,NaN,126.906070,37.442495
20860,20231106,07:34:00,NaN,127.059652,37.669789
27235,20231107,22:04:00,NaN,127.071881,37.645576
...,...,...,...,...,...
3069850,20231031,20:04:00,NaN,0.000000,0.000000
3069898,20231031,20:17:00,NaN,0.000000,0.000000
3070014,20231031,20:45:00,NaN,127.063964,37.645916
3070469,20231031,23:22:00,NaN,0.000000,0.000000


,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
3070523,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
3070524,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
3070525,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
3070526,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [37]:
# Find rows in df_reports_dropped with any missing values
df_reports_missing = df_reports_dropped[df_reports_dropped.isnull().any(axis=1)]

# Drop these rows from df_reports_dropped
df_reports_dropped = df_reports_dropped.drop(df_reports_missing.index)

# Display the DataFrame with missing rows and the cleaned DataFrame
print('missing: ', df_reports_missing.shape)
display(df_reports_dropped)

missing:  (27, 5)


,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
3070523,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
3070524,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
3070525,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
3070526,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [38]:
missing_values = df_reports_dropped.isnull().sum()
print(missing_values)

민원접수일     0
민원접수시간    0
주소        0
경도        0
위도        0
dtype: int64


In [53]:
df_reports_cleaned = df_reports_dropped.copy()
df_reports_cleaned.head()

,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286


In [40]:
pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [44]:
df_reports_cleaned.shape

(3064538, 5)

In [45]:
import holidays

df_reports_cleaned['민원접수일'] = pd.to_datetime(df_reports_cleaned['민원접수일'].astype(str), format='%Y%m%d')

# Load Korean public holidays for all relevant years
years = df_reports_cleaned['민원접수일'].dt.year.unique().tolist()
kr_holidays = holidays.KR(years=years)


In [46]:
# Define a function to classify each date
def classify_day(d):
    wd = d.weekday()  # Mon=0 … Sat=5, Sun=6
    if wd == 5:
        return 'Saturday'
    # treat Sunday or any official holiday as Holiday
    elif wd == 6 or d in kr_holidays:
        return 'Holiday'
    else:
        return 'Weekday'


In [47]:

df_reports_cleaned['요일'] = df_reports_cleaned['민원접수일'].apply(classify_day)
df_reports_cleaned['요일'].value_counts()

요일
Weekday     2027023
Holiday      546988
Saturday     490527
Name: count, dtype: int64

In [48]:
# sort by 민원접수일
df_reports_cleaned.sort_values(by='민원접수일', ascending=True, inplace=True)
# Reset the index
df_reports_cleaned.reset_index(drop=True, inplace=True)
df_reports_cleaned

,민원접수일,민원접수시간,주소,경도,위도,요일
0,2021-09-29,19:29:00,서울특별시 강서구 강서로15길 49,126.843247,37.532089,Weekday
1,2021-09-29,18:48:00,성북구 오패산로19길 34-5,127.033761,37.609537,Weekday
2,2021-09-29,18:47:00,장위로21다길 59-19 주소지 앞도로 외,127.045741,37.616406,Weekday
3,2021-09-29,18:47:00,서울특별시 강북구 오패산로30길 13,127.034685,37.613820,Weekday
4,2021-09-29,18:46:00,서울특별시 강서구 강서로18길 52-5,126.848703,37.534293,Weekday
...,...,...,...,...,...,...
3064533,2024-03-13,12:33:00,서울특별시 마포구 동교로 144,126.918137,37.555325,Weekday
3064534,2024-03-13,12:34:00,서울특별시 서초구 강남대로101안길 4,127.018191,37.516667,Weekday
3064535,2024-03-13,12:34:00,서울특별시 마포구 동교로12길 41-16,126.913712,37.552550,Weekday
3064536,2024-03-13,12:28:00,서울 강동구 성안로 11-5,127.128981,37.525097,Weekday


In [49]:
df_reports_cleaned.to_csv('reports.csv', index=False, encoding='utf-8-sig')

In [54]:
df_parks_cleaned.head()

,주소,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도,1시간 요금
0,강남구 개포동 126-2,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.06647710446097,37.477263060380004,2400.0
1,강남구 개포동 1266-0,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.04821844306556,37.48149637668632,2400.0
2,강남구 개포동 13-2,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.0793065206814,37.49493847412162,1200.0
3,강남구 개포동 567-23,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.0658354146965,37.47788845798715,2400.0
4,강남구 논현동 168-0,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.02629013795874,37.50817148532231,3600.0


In [ ]:
df_parks_cleaned.drop(columns=['주차장종류', '기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간'], inplace=True)
display(df_parks_cleaned.head())

/var/folders/pf/y0k140gn11d95ybm0dgtk6m80000gn/T/ipykernel_34922/1024365028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parks_cleaned.drop(columns=['주차장종류', '기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간'], inplace=True)


,주소,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도,1시간 요금
0,강남구 개포동 126-2,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.06647710446097,37.477263060380004,2400.0
1,강남구 개포동 1266-0,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.04821844306556,37.48149637668632,2400.0
2,강남구 개포동 13-2,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.0793065206814,37.49493847412162,1200.0
3,강남구 개포동 567-23,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.0658354146965,37.47788845798715,2400.0
4,강남구 논현동 168-0,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.02629013795874,37.50817148532231,3600.0


In [56]:
df_parks_cleaned.to_csv('parks.csv', index=False, encoding='utf-8-sig')